<a href="https://colab.research.google.com/github/YCCS-Summer-2023-DDNMA/project/blob/80-practice-with-neural-networks-mk/Michael_Kupferstein/nn_from_scratch/nnScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP training on MNIST

In [2]:
# todo: init MPL and add the predict
# todo: add data loading in PyTorch
# todo: add the training loop, loss fn

import numpy as np
import jax

In [10]:
seed = 0

def init_MLP(layer_widths,parent_key,scale=0.01):

  params = []
  keys = jax.random.split(parent_key,num=len(layer_widths)-1)

  for in_width,out_width,key in zip(layer_widths[:-1],layer_widths[1:],keys):
    weight_key, bias_key = jax.random.split(key)
    params.append([
            scale*jax.random.normal(weight_key,shape=(out_width,in_width)),
            scale*jax.random.normal(bias_key,shape=(out_width,))
            ]
    )

  return params

key = jax.random.PRNGKey(seed)
MLP_params = init_MLP([784,512,256,10],key)

print(jax.tree_map(lambda x:x.shape,MLP_params))

[[(512, 784), (512,)], [(256, 512), (256,)], [(10, 256), (10,)]]


# Visualizations

In [ ]:
# todo: visualize the MPL weight
# todo: visulaize emedding using t-SNE
# todo: dead neurons

# Parallelization